# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [54]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core import Environment

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, GridParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'amazon_review_prediction'

experiment=Experiment(ws, experiment_name)

ds = TabularDatasetFactory.from_delimited_files(path="https://github.com/iolucas/AMLE_P3/raw/main/amazon_reviews.csv") 

In [3]:
found = False
dataset_name = "Amazon Reviews"
dataset_descriptions = "Set of Amazon products text reviews together with their rating value ranging from 1 to 5."

if dataset_name in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[dataset_name] 

if not found:
        dataset = TabularDatasetFactory.from_delimited_files(path="https://github.com/iolucas/AMLE_P3/raw/main/amazon_reviews.csv") 
        dataset = ds.register(workspace=ws, name=dataset_name, description=dataset_descriptions)

In [4]:
df = ds.to_pandas_dataframe()
print(df.shape)
df.head()

(28332, 2)


,review,rating
0,I order 3 of them and one of the item is bad q...,3
1,Bulk is always the less expensive way to go fo...,4
2,Well they are not Duracell but for the price i...,5
3,Seem to work as well as name brand batteries a...,5
4,These batteries are very long lasting the pric...,5


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

In [6]:
#compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

In [21]:
!python train.py

usage: train.py [-h] --data-dir DATA_DIR --output-dir OUTPUT_DIR
                --use-stopwords --use-binary-count --ngram-range NGRAM_RANGE
train.py: error: the following arguments are required: --data-dir, --output-dir, --use-stopwords, --use-binary-count, --ngram-range


In [26]:
from azureml.core import Environment

#sklearn_env2 = Environment.from_conda_specification(name='sklearn-env2', file_path='conda_dependencies.yml')
sklearn_env = Environment.get(ws, "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [112]:
#No termination policy is used since scikitlearn methods does run iteratively and only logs primary metric once.
early_termination_policy = None#BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = GridParameterSampling({
    "--use-stopwords": choice(0,),
    "--use-binary-count": choice(0, 1),
    "--ngram-range": choice(1,)  
})

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      arguments=[
                          #"--dataset-name", dataset_name,
                          #"--output-dir", "outputs",
                          #"--use-stopwords",
                          #"--use-binary-count",
                          #"--ngram-range", 1
                      ],
                      environment=sklearn_env
                     )

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator, 
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

In [113]:
#TODO: Submit your experiment
hdr = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [115]:
RunDetails(hdr).show()
hdr.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_fc21e678-94c2-4897-9e19-c1ddaa0614d3',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-15T03:14:16.261332Z',
 'endTimeUtc': '2021-08-15T03:16:19.612466Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'bf9ec40c-d26c-4b23-8e9f-f0e598e9a31f',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.7517204870301747',
  'best_child_run_id': 'HD_fc21e678-94c2-4897-9e19-c1ddaa0614d3_1',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg154545.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_fc21e678-94c2-4897-9e19-c1ddaa0614d3/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [116]:
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)

print('\n Accuracy:', best_run_metrics['Accuracy'])

for i in range(0,len(parameter_values),2):
    print(parameter_values[i], parameter_values[i+1])

Best Run Id:  HD_fc21e678-94c2-4897-9e19-c1ddaa0614d3_1

 Accuracy: 0.7517204870301747
--use-stopwords 0
--use-binary-count 1
--ngram-range 1


In [119]:
best_run.download_file('nb_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [122]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='nb_model.pkl',                # Name of the registered model in your workspace.
                       model_path='./nb_model.pkl',  # Local file to upload and register as a model.
                       #model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       #model_framework_version='0.19.1',             # Version of scikit-learn used to create the model.
                       #description='Ridge regression model to predict diabetes progression.',
                       #tags={'area': 'diabetes', 'type': 'regression'}
                      )

#print('Name:', model.name)
#print('Version:', model.version)

Registering model nb_model.pkl


In [127]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory=".", entry_script="score.py", environment=sklearn_env)

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


In [148]:
service_name = 'my-sklearn-service'

service = model.deploy(ws, service_name, [model], overwrite=True, inference_config=inference_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-15 04:19:28+00:00 Creating Container Registry if not exists.
2021-08-15 04:19:28+00:00 Registering the environment.
2021-08-15 04:19:28+00:00 Use the existing image.
2021-08-15 04:19:28+00:00 Generating deployment configuration.
2021-08-15 04:19:29+00:00 Submitting deployment to compute.
2021-08-15 04:19:31+00:00 Checking the status of deployment my-sklearn-service..
2021-08-15 04:19:41+00:00 Checking the status of inference endpoint my-sklearn-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [156]:
import json


input_payload = json.dumps({
    'data': ["I loved this product", "awful experience."],
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)
print(output)

[5, 1]


TODO: In the cell below, print the logs of the web service and delete the service

In [157]:
print(service.get_logs())

2021-08-15T04:19:38,477664045+00:00 - rsyslog/run 
2021-08-15T04:19:38,479204705+00:00 - iot-server/run 
2021-08-15T04:19:38,677997625+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-15T04:19:39,076115585+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-15T04:19:40,977786934+00:00 - iot-server/finish 1 0
2021-08-15T04:19:41,077199595+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (12)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 37
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-08-15 04:19:45,682 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-08-15 04:19:45,683 | root | INFO | Starting up request id generator
2021-08-15 04:19:45,683 | root | INFO | Star

In [141]:
import requests

In [159]:
requests.post("http://1abcfca4-2d7f-4535-a5bf-b1e509bc386d.southcentralus.azurecontainer.io/score", json={
    "data":["i loved it", "fine product"]
}).text

'[5, 5]'